In [1]:
# import main libraries
import json
import requests
import pandas as pd
import numpy as np

In [2]:
# import all datasets
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [3]:
type(distances_stations_db)

dict

In [4]:
df_distance_stations_ntas = pd.read_csv("dataset/distance_stations_neighborhoods.csv", index_col="Unnamed: 0")
df_distance_stations = pd.DataFrame(distances_stations_db)
df_distance_stations_ntas

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,2123.18,2130.20,2104.71,2073.11,1896.78,1881.48,1925.06,1876.57,2030.47,1974.10,...,2449.03,2393.86,2396.74,2617.64,3444.35,3471.11,3761.95,3933.27,3559.47,2631.33
BX28,1894.39,1874.63,1935.64,2020.57,1809.95,1751.58,1728.54,1601.29,1843.17,1762.19,...,3168.88,3133.14,3051.15,3274.89,3596.90,3272.39,3250.70,3276.83,3037.71,3351.18
QN55,1872.67,1776.72,1811.35,1821.87,1646.27,1630.97,1674.55,1626.81,1779.96,1724.34,...,2113.36,2058.18,2061.06,2281.96,3108.67,3135.43,3426.27,3597.59,3223.79,2295.65
BK50,1532.61,1587.44,1614.04,1630.87,1548.42,1533.11,1576.70,1662.97,1682.11,1704.08,...,1726.71,1671.54,1674.41,1895.32,2722.02,2748.79,3039.63,3210.95,2837.15,1909.01
BX41,1774.19,1754.43,1815.43,1900.37,1689.74,1631.38,1608.33,1481.08,1722.97,1641.99,...,3048.68,3012.94,2893.91,3117.65,3439.67,3115.16,3093.46,3119.59,2880.48,3201.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BK93,1571.05,1625.88,1652.48,1669.31,1586.86,1571.56,1615.15,1701.41,1720.55,1742.52,...,1753.73,1705.62,1706.59,1922.34,2756.10,2775.81,3066.65,3243.13,2869.33,1936.03
BK73,723.32,730.33,704.84,673.24,496.91,481.61,525.20,611.46,630.60,652.57,...,1791.46,1750.57,1744.32,1960.06,2799.28,2813.53,2988.50,3097.89,2775.52,1973.75
MN28,446.34,463.45,452.35,432.20,240.24,101.92,243.76,387.74,406.88,428.85,...,1774.52,1738.78,1727.38,1943.12,2778.55,2619.51,2747.46,2856.85,2534.48,1956.81
BK40,657.93,767.28,739.37,756.19,906.25,1014.20,1085.85,1121.00,902.06,975.73,...,1058.62,1017.73,1011.48,1227.22,2066.44,2080.69,2371.53,2548.01,2174.21,1240.91


In [5]:
# define vehicle types
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]
# define vehicle type quantities
vehicle_quantities = [197, 143, 5, 8, 1]

In [6]:
# create two dictionaries for the serviceability of the stations
dfs_nta_serviceability = {}
serviceability_distances ={}

# iterate all vehicle types
for vehicle_type in vehicle_types:
    # create datframes of the same shape of the nta-stations distances, all false 
    dfs_nta_serviceability[vehicle_type] = df_distance_stations_ntas.copy()
    dfs_nta_serviceability[vehicle_type].loc[:] = True

    serviceability_distances[vehicle_type] = 0

In [7]:
def get_coverage_overlaps(df):
    # check if there is at leas an overalp on the coverage
    def check_coverage_overlap(riga1, riga2):
        # return true if there is at leas an overalp on the coverage, false otherwise
        return any(riga1 & riga2)

    # transpose dataframe for checking the stations as rows
    df = df.T

    # create a results list
    results = [False] * len(df)
    # convert the list to a series
    results_series = pd.Series(results, index=df.index)

    # iter each row of the dataframe
    for index_1, row_1 in df.iterrows():
        # check if the row wasn't already checked
        if results_series[index_1] == False:
            # iter each row of the dataframe
            for index_2, row_2 in df.iterrows():
                # chech if the second row is after the first one, and check if it wasn't already checked
                if (index_2 > index_1) and (results_series[index_2] == False):
                    # checl the overlap for the couple of rows
                    is_overlap = check_coverage_overlap(row_1, row_2)
                    # store the result
                    results_series[index_2] = is_overlap 
    
    # return the series with the result for each row
    return results_series

In [8]:
def set_serviceability(distance_threshold, df_nta_serviceability):
    # interate nta-stations distances dataframe
    for (nta, distances) in df_distance_stations_ntas.iterrows():
        # iterate stations' distances from the neighborhood
        for station, distance_st_nta in distances.items():
            # check if the distance between the nta and the station is greater then the threshold
            if distance_st_nta > distance_threshold:
                # set the neighborhood as unservable by that station
                df_nta_serviceability.loc[nta, station] = False
    
    # return the serviceability dataframe
    return df_nta_serviceability

In [9]:
# iterate all vehicle types and relative quantities
for vehicle_type, vehicle_quantity in zip(vehicle_types[:-1], vehicle_quantities[:-1]):
    
    # set an upper limit for the serviceability distance threshold
    serviceability_threshold = 910
    # set an initial value for the serviceability areas counter
    serviceability_areas = 0
    # make a copy of the original serviceability dataframe (just for the first while check)
    vehicle_serviceability = dfs_nta_serviceability[vehicle_type].copy()

    # iterate until a neighborhood cannot be served by any station and until the serviceability areas are less then the vehicle quantity
    while vehicle_serviceability.any(axis=1).all() and vehicle_serviceability.any(axis=0).all() and serviceability_areas <= vehicle_quantity:
        # low the serviceability distance threshold
        serviceability_threshold -= 1

        # get a copy of the original dataframe
        vehicle_serviceability = dfs_nta_serviceability[vehicle_type].copy()
        # set the serviceability based on the threshold
        vehicle_serviceability = set_serviceability(serviceability_threshold, vehicle_serviceability).copy()

        # get the overlapping areas
        duplicates = get_coverage_overlaps(vehicle_serviceability)

        # count the non-overlapping areas
        serviceability_areas = (~duplicates).sum()

    # get a copy of the original dataframe
    vehicle_serviceability = dfs_nta_serviceability[vehicle_type].copy()
    # set the serviceability based on the last threshold + 1
    vehicle_serviceability = set_serviceability(serviceability_threshold + 1, vehicle_serviceability).copy()

    # store the serviceability table
    dfs_nta_serviceability[vehicle_type] = vehicle_serviceability
    # store the serviceability distance
    serviceability_distances[vehicle_type] = serviceability_threshold + 1
    # print the serviceability distance
    print(vehicle_type + ": " + str(serviceability_threshold + 1))
    

engine: 754
ladder: 754
rescue: 902
squad: 754


In [10]:
for df in dfs_nta_serviceability:
    dfs_nta_serviceability[df].to_csv("dataset/serviceability/" + df + "_ntas_serviceability.csv")